# Part-of-speech-tagging with *stanza*



This is a Python notebook for pos-tagging that is meant to be used in Google Colab.

The following code will use the Python library *stanza* to tokenize and annotate text.

Further information on *stanza*:

*   How to use *stanza*: https://stanfordnlp.github.io/stanza/index.html
*   The paper on *stanza*: https://aclanthology.org/2020.acl-demos.14/

This code is written specifically to ignore XML-tags that is to print them unchanged and only tokenize and annotate raw text. This is ideal for use with the IMS Open Corpus Workbench where XML-tags provide metadata on each text and should not be annotated.

In theory, you can use this notebook locally on your computer with the necessary dependencies (e.g. Jupyter Notebooks). In that case, you cannot use any commands specific to Google Colab. You will likely also not have a suitable GPU that will make your code run faster.

## Use the GPU
Be sure to set the runtime in Google Colab to a GPU, so the code will run faster. This makes a huge difference with large files!

Google Colab will not select a GPU by default. To change to a GPU in Google Colab, click on **Runtime** above, select **Change runtime type** and select a GPU. Your notebook will disconnect and reconnect, so you may have to rerun cells.

Google Colab may limit the number of times you can use a GPU per day, but this should not be a problem, as the code works on the default settings (CPU) as well.

## Install and import *stanza*


...and other necessary modules.

In [1]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 38.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [2]:
import stanza
import re
import os

## Download the language model for English

In [3]:
stanza.download("en")
print("Language model downloaded")
nlp = stanza.Pipeline("en", processors="tokenize, lemma, pos", verbose=False, use_gpu=True)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


Language model downloaded


For other languages and their corresponding codes (lcode), please see this list: https://stanfordnlp.github.io/stanza/performance.html
For example, for German, replace "en" with "de" or, for Dutch, use "nl".

## Verify that you're using a GPU

Depending on whether the GPU is active or not, the output of this cell should be ```True``` or ```False```.

In [ ]:
import torch

torch.cuda.is_available()

True

## Test the tagger with shorter texts

You may find this useful to gain an impression of how well the tagger performs with your type of text – a song by Snoop Dogg and Dr. Dre, for example.

In [4]:
# Run this cell to set the sample text

sample = """

And even when I was close to defeat, I rose to my feet
My life's like a soundtrack I wrote to the beat
Treat rap like Cali weed: I smoke 'til I'm 'sleep
Wake up in the A.M., compose a beat
I bring the fire 'til you're soakin' in your seat

"""

Stanza will print each **token**, the **PENN Treebank tag** (xpos), the corresponding **Universal Dependencies tag** (pos), and the **lemma** (lemma) in all lowercase letters on a new line.

See here for further information on the tagsets:

*   **Universal Dependencies Tagset**: https://universaldependencies.org/u/pos
*   **PENN Treebank Tagset**: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html


The output will be a tab-separated list, with each token on a separate line.


In [18]:
# Run this to cell to tag the sample text

def tag_text(text):
  # pattern to split XML tags and non-tags
  # group 1: XML tags, group 2: non-tag text
  parts = re.findall(r'(<[^>]+>)|([^<]+)', text)


  for tag, text in parts:
      if tag:
          # it's an XML tag, so write it unchanged
          print(tag + "\n")
      elif text:
          # it's text content, so lemmatize, clean and write
          doc = nlp(text)

          for sentence in doc.sentences:
              for word in sentence.words:
                  print(f"{word.text}\t{word.xpos}\t{word.pos}\t{word.lemma.lower()}\n")


tag_text(sample)

And	CC	CCONJ	and

even	RB	ADV	even

when	WRB	ADV	when

I	PRP	PRON	i

was	VBD	AUX	be

close	JJ	ADJ	close

to	IN	ADP	to

defeat	NN	NOUN	defeat

,	,	PUNCT	,

I	PRP	PRON	i

rose	VBD	VERB	rise

to	IN	ADP	to

my	PRP$	PRON	my

feet	NNS	NOUN	foot

My	PRP$	PRON	my

life	NN	NOUN	life

's	VBZ	AUX	be

like	UH	INTJ	like

a	DT	DET	a

soundtrack	NN	NOUN	soundtrack

I	PRP	PRON	i

wrote	VBD	VERB	write

to	IN	ADP	to

the	DT	DET	the

beat	NN	NOUN	beat

Treat	NN	NOUN	treat

rap	NN	NOUN	rap

like	IN	ADP	like

Cali	NNP	PROPN	cali

weed	NN	NOUN	weed

:	:	PUNCT	:

I	PRP	PRON	i

smoke	VBP	VERB	smoke

'	``	PUNCT	'

til	IN	SCONJ	till

I	PRP	PRON	i

'm	VBP	AUX	be

'	``	PUNCT	'

sleep	VB	VERB	sleep

Wake	VB	VERB	wake

up	RP	ADP	up

in	IN	ADP	in

the	DT	DET	the

A.M.	NN	NOUN	a.m.

,	,	PUNCT	,

compose	VB	VERB	compose

a	DT	DET	a

beat	NN	NOUN	beat

I	PRP	PRON	i

bring	VBP	VERB	bring

the	DT	DET	the

fire	NN	NOUN	fire

'	``	PUNCT	'

til	IN	SCONJ	till

you	PRP	PRON	you

're	VBP	AUX	be

soakin	VBG	VERB	soakin

'	''	PU

## Use the tagger with a text file



### Option 1: Upload the file directly to this Google Colab notebook

Click on the folder icon on the lefthand side, then click on the file icon with the upward arrow to upload a file from your computer.

Be aware that all your files uploaded to this project will be deleted once this runtime disconnects, e.g. if it has been inactive for a while.

In [ ]:
# this command shows you the current working directory
print("You're currently in the folder:")
!pwd

#this command lists all the file in your current working directory
print(f"The current contents of the folder are:")
!ls

You're currently in the folder:
/content
The current contents of the folder are:
drive  sample_data  swift.vrt  swift.xml  test.txt  test.vrt


When inserting the name of your file in the cell below, please include the file extension as well. For a file named "test.txt", the extension would be ".txt".

Make sure that the file name is correct and does not contain whitespace on either side.

In [40]:
# Insert the name of your file inside the quotation marks
input_file = "test.txt"

In [41]:
input_file = f"/content/{input_file}"

name, ext = os.path.splitext(input_file)
output_file = name + ".vrt"

### Option 2: Connect this notebook to your Google Drive account

If you run this cell, a window should open, asking for permission. If you grant it, another window will have you select the Gooogle Drive account and specify which permissions to grant. You need to allow the notebook to open, read, edit and delete files. This will likely be only one tick box.

If everything worked, the output of the cell should say: ```Mounted at /content/drive```.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


If your file is saved in your Google Drive, the notebook can access it with the following cells.

**The file itself will not be changed!** It will only be opened in read-mode "r", meaning its contents will be read. The notebook will produce a new file with the tagged text.

When inserting the name of your file in the cell below, please include the file extension as well. For a file named "test.txt", the extension would be ".txt". Make sure that the file name is correct and does not contain whitespace on either side.


In [38]:
# Insert the name of your file inside the quotation marks
input_file = "test.txt"

In [43]:
input_file = f"/content/drive/My Drive/{input_file}"

name, ext = os.path.splitext(input_file)
output_file = name + ".vrt"

## Tag your file

With Google Colab set to use the GPU, this code will approximately need 6-7 minutes to tokenize and annotate a file with over 100.000 tokens.

Be aware that the following code will **automatically download** the created file to your computer, unless the indicated line is commented out. It is advised that the file be immediately downloaded, because all files will be deleted as soon as the runtime disconnects (e.g. if the notebook has been inactive for a while).

The downloaded file will have the name of the original input file with the extension changed to ".vrt" (VeRTicalized file). This is just a normal text file that can be opened with any editor. Find out more here: https://fedora.clarin-d.uni-saarland.de/teaching/Comparing_Corpora_Tutorials/Tutorial_VRT.html

As said before, Stanza will print each **token**, the **PENN Treebank tag** (xpos), the corresponding **Universal Dependencies tag** (pos), and the **lemma** (lemma) in all lowercase letters on a new line.


In [42]:
# Run this cell to tag the file

from google.colab import files


def tag_file(input_file, output_file):

    with open(input_file, "r") as input_file:
      content = input_file.read()

    # pattern to split XML tags and non-tags
    # group 1: XML tags, group 2: non-tag text
    parts = re.findall(r'(<[^>]+>)|([^<]+)', content)


    with open(output_file, "w", encoding="utf-8") as out:
        for tag, text in parts:
            # print(tag, text)
            if tag:
                # it's an XML tag, so write it unchanged
                out.write(tag + "\n")
            elif text:
                # it's text content, so lemmatize, clean and write
                doc = nlp(text)

                for sentence in doc.sentences:
                    for word in sentence.words:
                        out.write(
                            f"{word.text}\t{word.xpos}\t{word.pos}\t{word.lemma.lower()}\n")

    return f"Lemmatized text written to {output_file}."


print(tag_file(input_file=input_file, output_file=output_file))

# download the output file to your computer
files.download(output_file)

print(f"{output_file} downloaded.")

Lemmatized text written to /content/test.vrt.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/content/test.vrt downloaded.
